In [1]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [2]:
!pip install sklearn-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 12.5 MB/s eta 0:00:00


In [11]:
import os
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import json

def space_label(sentence):
  #문장 라벨 리스트에 저장
  label = []
  for i, char in enumerate(sentence):
    if i == 0 or sentence[i-1] == ' ':  # 문장의 첫 글자거나 이전 문자가 공백인 경우
      label.append('B')
    else:
      label.append('I')

  return label

#문장 음절 리스트에 저장
def split_sentence(sentence):
  sentence = sentence.replace(" ", "")
  return list(sentence)

def delete_content(entity):
  del(entity["title"])
  del(entity["subtitle"])
  del(entity["board"])
  del(entity["writer"])
  del(entity["write_date"])
  del(entity["url"])
  del(entity["source_site"])

In [37]:
def set_sentence(folder_path, num, data):
  cnt = 0
  file_list = os.listdir(folder_path)
  for file in file_list:
    file_path = folder_path + "/" + file

    if cnt<num:
      with open(file_path, "r") as f:
        json_data = json.load(f)
      named_entity = json_data["named_entity"]
      for entity in named_entity:
        if cnt<num:
          for content in entity["content"]:
            if cnt<num:
              del(content["labels"])
              eumjeol_sequence = split_sentence(content["sentence"])
              label_sequence = space_label(content["sentence"])
              if len(eumjeol_sequence) == len(label_sequence):
                data.append((eumjeol_sequence, label_sequence))
                cnt = cnt + 1

In [38]:
datas = []

# 파일 경로
set_sentence("/gdrive/파일 경로/dataset/산업", 850, datas)
set_sentence("/gdrive/파일 경로/dataset/연예", 840, datas)
set_sentence("/gdrive/파일 경로/dataset/경제", 830, datas)
set_sentence("/gdrive/파일 경로/dataset/정치", 800, datas)
set_sentence("/gdrive/파일 경로/dataset/여행레저", 670, datas)
set_sentence("/gdrive/파일 경로/dataset/사회일반", 640, datas)
set_sentence("/gdrive/파일 경로/dataset/취미", 800, datas)
set_sentence("/gdrive/파일 경로/dataset/건강", 580, datas)
set_sentence("/gdrive/파일 경로/dataset/사건사고", 560, datas)
set_sentence("/gdrive/파일 경로/dataset/스포츠", 540, datas)
set_sentence("/gdrive/파일 경로/dataset/라이프스타일", 530, datas)
set_sentence("/gdrive/파일 경로/dataset/지역", 490, datas)
set_sentence("/gdrive/파일 경로/dataset/교육", 490,datas)
set_sentence("/gdrive/파일 경로/dataset/여성복지", 430,datas)
set_sentence("/gdrive/파일 경로/dataset/국제", 420, datas)
set_sentence("/gdrive/파일 경로/dataset/문화", 380, datas)
set_sentence("/gdrive/파일 경로/dataset/IT_과학", 360, datas)

In [47]:
print("데이터 개수 : "+str(len(datas)))
number_of_train_datas = int(len(datas)*0.9)

train_datas = datas[:number_of_train_datas]
test_datas = datas[number_of_train_datas:]

for d in datas:
  print(d[0],)
  print(d[1])
  print()

print("train_datas 개수 : " + str(len(train_datas)))
print("test_datas 개수 : " + str(len(test_datas)))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['1', '점', ')', '.']
['B', 'I', 'I', 'I']

['2', '점', ')', '.']
['B', 'I', 'I', 'I']

['3', '점', ')', '.']
['B', 'I', 'I', 'I']

['1', '점', ')', '.']
['B', 'I', 'I', 'I']

['.', '.', '.', '.']
['B', 'I', 'I', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['왜', '?']
['B', 'I']

['#', '.']
['B', 'I']

['#', '.']
['B', 'I']

['#', '.']
['B', 'I']

['━', '.']
['B', 'I']

['1', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['2', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['3', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['4', '.']
['B', 'I']

['━', '.']
['B', 'I']

['━', '.']
['B', 'I']

['5', '.']
['B', 'I']

['완', '벽', '주', '의', '자', '다', '.']
['B', 'I', 'I', 'I', 'I', 'I', 'I']

['━', '.']
['B', 'I']

['━', '.'

In [40]:
def sent2feature(eumjeol_sequence):
  features = []
  sequence_length = len(eumjeol_sequence)
  for index, eumjeol in enumerate(eumjeol_sequence):
      feature = { "BOS":False, "EOS":False, "WORD":eumjeol, "IS_DIGIT":eumjeol.isdigit() }

      if(index==0):
          feature["BOS"]=True
      elif(index==sequence_length-1):
          feature["EOS"]=True

      if(index-1>=0):
          feature["-1_WORD"] = eumjeol_sequence[index-1]
      if(index-2>=0):
          feature["-2_WORD"] = eumjeol_sequence[index-2]

      if(index+1<=sequence_length-1):
          feature["+1_WORD"] = eumjeol_sequence[index+1]
      if(index+2<=sequence_length-1):
          feature["+2_WORD"] = eumjeol_sequence[index+2]

      features.append(feature)

  return features

train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

In [42]:
crf = sklearn_crfsuite.CRF()
try:
  crf.fit(train_x, train_y)
except AttributeError:
  pass

In [46]:
def show_predict_result(test_datas, predict):
  for index_1 in range(len(test_datas)):
      eumjeol_sequence, correct_labels = test_datas[index_1]
      predict_labels = predict[index_1]

      correct_sentence, predict_sentence = "", ""
      for index_2 in range(len(eumjeol_sequence)):
          if(index_2 == 0):
              correct_sentence += eumjeol_sequence[index_2]
              predict_sentence += eumjeol_sequence[index_2]
              continue

          if(correct_labels[index_2] == "B"):
              correct_sentence += " "
          correct_sentence += eumjeol_sequence[index_2]

          if (predict_labels[index_2] == "B"):
              predict_sentence += " "
          predict_sentence += eumjeol_sequence[index_2]

      print("정답 문장 : " + correct_sentence)
      print("출력 문장 : " + predict_sentence)
      print()

predict = crf.predict(test_x)

print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, predict)))
print()

print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

for i in range(10):
  print(test_datas)
  print(train_datas)

show_predict_result(test_datas[:10], predict[:10])

Output hidden; open in https://colab.research.google.com to view.